<a href="https://colab.research.google.com/github/kkamalk/podcast_reviews/blob/main/Inferential_Statistical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1.Podcast_Analysis**


##**Introduction**

 2 million podcast reviews for 100k podcasts, updated monthly. This dataset is intended to aid in analysis of text feedback and review data.

## **Objectives for this Part**

Practice working with SQLite datasets.

Practice performing EDA.

Practice applying statistical inference procedures.

Practice visualizing data with Matplotlib & Seaborn.

Practice reading data, performing queries and filtering data using SQL and Pandas.

## **Requirements**

Load the data using SQLite and Pandas.

Perform exploratory data analysis. This should include creating 

statistical summaries and charts, testing for anomalies, checking for correlations and other relations between variables, and other EDA elements.

Perform statistical inference. This should include defining the target population, forming multiple statistical hypotheses and constructing confidence intervals, setting the significance levels, conducting z or t-tests for these hypotheses.

Provide clear explanations in your notebook. Your explanations should inform the reader what you are trying to achieve, what results did you get, and what these results mean.

Provide suggestions about how your analysis can be improved. 



## **Evaluation Criteria**

Adherence to the requirements. How well did you meet the requirements?
Depth of your analysis. Did you just skim the surface or did you explored the dataset in depth?

Visualization quality. Did you use charts effectively to visualize patterns in the data? Are your visualizations properly labeled? Did you use colors effectively? Did you adhere to the principle of proportional ink?

Code quality. Was your code well-structure? Did you use the appropriate levels of abstraction? Did you remove commented out and unused code? Did you adhere to the PEP8?

Code performance. Did you use the suitable algorithms and data structures to solve the problems?

# **2.Libraries and Loading Data**

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
pd.set_option('display.max_columns', 500)

In [2]:
from google.colab import drive
drive.mount("/content/drive")

con = sqlite3.connect("/content/drive/MyDrive/database.sqlite")

Mounted at /content/drive


In [3]:
pd.read_sql("SELECT * FROM sqlite_master WHERE TYPE = 'table' ", con)

,type,name,tbl_name,rootpage,sql
0,table,runs,runs,2,CREATE TABLE runs (\n run_at text not n...
1,table,podcasts,podcasts,3,CREATE TABLE podcasts (\n podcast_id te...
2,table,categories,categories,5,CREATE TABLE categories (\n podcast_id ...
3,table,reviews,reviews,7,CREATE TABLE reviews (\n podcast_id tex...


In [4]:
df_categories = pd.read_sql_query("SELECT * from categories", con)
df_podcasts= pd.read_sql_query("SELECT * from podcasts", con)
df_reviews = pd.read_sql_query("SELECT * from reviews", con)
df_runs = pd.read_sql_query("SELECT * from runs", con)

**Checking the Tables:**

In [5]:
df_categories.head()

,podcast_id,category
0,c61aa81c9b929a66f0c1db6cbe5d8548,arts
1,c61aa81c9b929a66f0c1db6cbe5d8548,arts-performing-arts
2,c61aa81c9b929a66f0c1db6cbe5d8548,music
3,ad4f2bf69c72b8db75978423c25f379e,arts
4,ad4f2bf69c72b8db75978423c25f379e,arts-design


In [6]:
df_podcasts.head()

,podcast_id,itunes_id,slug,itunes_url,title
0,a00018b54eb342567c94dacfb2a3e504,1313466221,scaling-global,https://podcasts.apple.com/us/podcast/scaling-...,Scaling Global
1,a00043d34e734b09246d17dc5d56f63c,158973461,cornerstone-baptist-church-of-orlando,https://podcasts.apple.com/us/podcast/cornerst...,Cornerstone Baptist Church of Orlando
2,a0004b1ef445af9dc84dad1e7821b1e3,139076942,mystery-dancing-in-the-dark,https://podcasts.apple.com/us/podcast/mystery-...,Mystery: Dancing in the Dark
3,a00071f9aaae9ac725c3a586701abf4d,1332508972,kts-money-matters,https://podcasts.apple.com/us/podcast/kts-mone...,KTs Money Matters
4,a000a500f06555f81220c3eb641aded7,1544900779,word-on-the-street-w-dreak-swift,https://podcasts.apple.com/us/podcast/word-on-...,Word on the Street w/ Dreak Swift


In [7]:
df_reviews.head()

,podcast_id,title,content,rating,author_id,created_at
0,c61aa81c9b929a66f0c1db6cbe5d8548,really interesting!,Thanks for providing these insights. Really e...,5,F7E5A318989779D,2018-04-24T12:05:16-07:00
1,c61aa81c9b929a66f0c1db6cbe5d8548,Must listen for anyone interested in the arts!!!,Super excited to see this podcast grow. So man...,5,F6BF5472689BD12,2018-05-09T18:14:32-07:00
2,ad4f2bf69c72b8db75978423c25f379e,nauseatingly left,"I'm a liberal myself, but its pretty obvious a...",1,1AB95B8E6E1309E,2019-06-11T14:53:39-07:00
3,ad4f2bf69c72b8db75978423c25f379e,Diverse stories,I find Tedx talks very inspirational but I oft...,5,11BB760AA5DEBD1,2018-05-31T13:08:09-07:00
4,ad4f2bf69c72b8db75978423c25f379e,👍👍👍👍,"I love this podcast, it is so good.",5,D86032C8E57D15A,2019-06-19T13:56:05-07:00


In [8]:
df_runs.head()

,run_at,max_rowid,reviews_added
0,2021-05-10 02:53:00,3266481,1215223
1,2021-06-06 21:34:36,3300773,13139
2,2021-07-02 18:04:55,3329699,11561
3,2021-08-01 17:54:42,3360315,11855
4,2021-09-02 18:00:30,3390165,11714


**Basic Checks**

In [9]:
df_categories.describe()

,podcast_id,category
count,211760,211760
unique,109602,110
top,be905117ff9b3a13b1a065801241903b,society-culture
freq,8,18698


Table categories consists of two features, representing podcast ID and the category. 

Trying to find out any missing and duplicate values

In [10]:
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211760 entries, 0 to 211759
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   podcast_id  211760 non-null  object
 1   category    211760 non-null  object
dtypes: object(2)
memory usage: 3.2+ MB


Findings:

There are no missing values

In [11]:
df_categories.duplicated().sum()

0

Findings:

There are no duplicated  values

I am trying to see what are the category types

In [12]:
pd.set_option('display.max_rows', 150)
pd.read_sql_query('''
                   SELECT DISTINCT category 
                   FROM categories
                   ''', con)
        
        
        

,category
0,arts
1,arts-performing-arts
2,music
3,arts-design
4,education
5,society-culture
6,arts-visual-arts
7,leisure
8,leisure-hobbies
9,arts-books


There are 110 unique categories in a table.However most of them are similiar. 
Therefore below categories are narrowed down by combining similiar ones ito one.



In [13]:
#write alterntive
query = """
select distinct(category_updated)
from(
  select *, 
   (case
      when category like "%art%" then 'Arts'
      when category like "%business%" then 'Business'
      when category like "%culture%" then 'Culture'
      when category = "buddhism" then 'Religion'
      when category = "hinduism" then 'Religion'
      when category = "islam" then 'Religion'
      when category = "christianity" then 'Religion'
      when category = "judaism" then 'Religion'
      when category = "religion-spirituality" then 'Religion'
      when category like "%society%" then "Society"
      when category like "%Fict%" then "Fiction"
      when category like "%Edu%" then "Education"
      when category like "%Health%" then "Health"
      when category like "%Tv%" then "Tv"
      when category like "%Comedy%" then "Comedy"
      when category like "%Kids%" then "Kids"
      when category like "%Leisure%" then "Leisure"
      when category like "%News%" then "News"
      when category like "%Music%" then "Music"
      when category like "%Science%" then "Science"
      when category like "%sport%" then "sports"
        else upper(substr(category, 1, 1)) || substr(category, 2)
   end) AS category_updated
  from categories
)
"""
df=pd.read_sql(query, con)
df

,category_updated
0,Arts
1,Music
2,Education
3,Culture
4,Leisure
5,Technology
6,Fiction
7,True-crime
8,Health
9,History


Findings:

After the extraction there exist only 20 different categories, each representing a different field.

In [14]:
df

,category_updated
0,Arts
1,Music
2,Education
3,Culture
4,Leisure
5,Technology
6,Fiction
7,True-crime
8,Health
9,History


**podcasts basic checks**

In [15]:
df_podcasts.sample(5)

,podcast_id,itunes_id,slug,itunes_url,title
46148,c8615d3fdd350e98abb0e30f52b55c2f,1342470293,another-mans-nostalgia-a-90s-and-00s-podcast,https://podcasts.apple.com/us/podcast/another-...,Another Man's Nostalgia - A 90s and 00s Podcast
37649,c0eae2ad5d957889a7eb752b4a9f135b,1374921534,bamb-the-pod,https://podcasts.apple.com/us/podcast/bamb-the...,BAMB! The Pod
21763,b30eda4170d438ae034e55cfb8c210ad,1573281689,young-entrepreneur-mindset-podcast,https://podcasts.apple.com/us/podcast/young-en...,Young Entrepreneur Mindset Podcast
8368,a75660078cbc91c090a98559b5c402f2,1492922725,learn-american-english-with-this-guy,https://podcasts.apple.com/us/podcast/learn-am...,Learn American English With This Guy
5084,a46f5114b0d3e234f259da78a3bb9074,1320979114,shanghai-chronicles,https://podcasts.apple.com/us/podcast/shanghai...,Shanghai Chronicles


In [16]:
df_podcasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109602 entries, 0 to 109601
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   podcast_id  109602 non-null  object
 1   itunes_id   109602 non-null  int64 
 2   slug        109602 non-null  object
 3   itunes_url  109602 non-null  object
 4   title       109602 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.2+ MB


Podcasts does not have any missing values.

In [17]:
df_podcasts[df_podcasts.title.astype(str).str.contains('None')]


,podcast_id,itunes_id,slug,itunes_url,title
13670,ac086204f34c0b65f93c460027cd1fad,1558021437,none-of-this-is-reel,https://podcasts.apple.com/us/podcast/none-of-...,None of This is Reel
15073,ad3316bd30d07b5878f2f61b3b951ff2,1337885213,dungeon-master-of-none,https://podcasts.apple.com/us/podcast/dungeon-...,Dungeon Master of None
16113,ae140141b4ae629223ff4aa0497a7f82,1347751328,nonexclusive,https://podcasts.apple.com/us/podcast/nonexclu...,Nonexclusive
31089,bb2396c22f4b4b919321b6f40d0d9427,1292426978,none-sense,https://podcasts.apple.com/us/podcast/none-sen...,None Sense
33140,bd0117a0c07adde7f444b8e2eb073955,1451821858,none-of-your-business,https://podcasts.apple.com/us/podcast/none-of-...,None Of Your Business
38469,c19de4a016b98d5fe91af3f863eba319,903699599,thus-spake-zarathustra-book-for-all-none-by-fr...,https://podcasts.apple.com/us/podcast/thus-spa...,Thus Spake Zarathustra: A Book for All and Non...
43479,c60c77815e6782f22e9f49da11584efd,593751261,the-none-to-run-podcast,https://podcasts.apple.com/us/podcast/the-none...,The None to Run Podcast
48307,ca3d2d0cf4587058a25ef043664340fc,1514036779,none-of-your-business-podcast,https://podcasts.apple.com/us/podcast/none-of-...,None Of Your Business Podcast
55519,d07ae5e7ab84921310eb560790346065,1439588586,none-of-this-is-real,https://podcasts.apple.com/us/podcast/none-of-...,None of This is Real
58165,d2cc76a7476b1e0b22255ba3408ee95b,1433153093,none-of-that,https://podcasts.apple.com/us/podcast/none-of-...,None of That


In [18]:
df_podcasts.describe()

,itunes_id
count,1.096020e+05
mean,1.308588e+09
std,3.576728e+08
min,7.332927e+07
25%,1.225780e+09
50%,1.467143e+09
75%,1.528596e+09
max,1.658655e+09


checking any duplicate values in the podcasts data

In [19]:
df_podcasts.duplicated().any()

False

In [20]:
def count_unique_values(df):
    return df.nunique()

df = pd.DataFrame(df_podcasts)
print(count_unique_values(df))


podcast_id    109602
itunes_id     109602
slug          108503
itunes_url    109602
title         108857
dtype: int64


It appears that there may be duplicates present in the 'slug' and 'title' columns of the DataFrame. To further investigate, I will examine if these duplicates indicate that certain podcasts have been included multiple times in the dataset or if they simply indicate that certain podcasts have similar 'slug' and 'title' values but different overall content.

In [21]:
duplicate_slugs = df_podcasts[df.duplicated(subset='slug', keep=False)]['slug']
duplicate_slugs.value_counts()


lets-talk-about-it       9
no-title                 8
in-the-loop              6
whats-next               6
off-the-cuff             6
                        ..
what-could-go-right      2
real-talk-with-raquel    2
song-story               2
shower-thoughts          2
to-the-point             2
Name: slug, Length: 945, dtype: int64

In [22]:
duplicate_slugs.value_counts()['off-the-record']


4

In [23]:
multiple_slugs = df_podcasts.query("slug == 'in-the-loop'")[['slug', 'itunes_url','title']]
multiple_slugs


,slug,itunes_url,title
36557,in-the-loop,https://podcasts.apple.com/us/podcast/in-the-l...,In The Loop
51645,in-the-loop,https://podcasts.apple.com/us/podcast/in-the-l...,In the Loop
52350,in-the-loop,https://podcasts.apple.com/us/podcast/in-the-l...,In The Loop
61712,in-the-loop,https://podcasts.apple.com/us/podcast/in-the-l...,In the Loop
74918,in-the-loop,https://podcasts.apple.com/us/podcast/in-the-l...,In the Loop
100681,in-the-loop,https://podcasts.apple.com/us/podcast/in-the-l...,In The Loop


Findings:

the Two podcasts have the same 'slug' or 'title', they may refer to different instances of podcasts with the same name. For example, they could be produced by different authors or cover different subjects. It's also possible that they are different episodes of the same podcast series. we also further investigated the duplicate entries and it is false


**Reviews basic checks**

In [24]:
df_reviews.sample(5)

,podcast_id,title,content,rating,author_id,created_at
1553221,c856ada8366bba0ef7825a3c004ccedf,Breath of fresh air,Fun and refreshing conversations amongst frien...,5,0D426A179820EC3,2020-09-14T18:54:58-07:00
851677,e6edbb3b09ab608dd57d787d28980e96,Meritorious,"Hope, puzzlement and beauty is found here. Truly.",5,565CD61B31EC3D2,2019-11-27T11:43:24-07:00
1208394,b628854e3917fdd2da020c6c2a038ddd,Natalie G is a Tudor treasure,She is such a gentle and generous host... alwa...,5,CB96FA9511A152D,2021-04-18T10:34:58-07:00
213727,d86cbec17e5cffe537e58b3cb273714f,Peace to you.,I am so thankful for weekly reminders of hope ...,5,02B5D503216913D,2018-11-08T18:41:38-07:00
1478812,bbe16e93e51a83b8bfbce3a2417bc7ac,A Gem,Great Podcast. Whenever I’m curious about some...,5,88FFF6DC5A3E12A,2021-02-01T06:11:46-07:00


In [25]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2025333 entries, 0 to 2025332
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   podcast_id  object
 1   title       object
 2   content     object
 3   rating      int64 
 4   author_id   object
 5   created_at  object
dtypes: int64(1), object(5)
memory usage: 92.7+ MB


We can see that all of the columns exept of 'rating' is categorial. Rating is of numeric type.

Lets find out any duplicates found

In [26]:
df_reviews.duplicated().sum()

652

We can see that there are 652 duplicated entries

In [27]:
df_reviews[df_reviews.duplicated(keep=False)]

,podcast_id,title,content,rating,author_id,created_at
3269,d31472739beeec4c8593810f7c158ad5,Great storyline,"Love this exciting story, delving into mysteri...",5,534676FEA811BBA,2017-07-13T22:05:51-07:00
3275,d31472739beeec4c8593810f7c158ad5,Great storyline,"Love this exciting story, delving into mysteri...",5,534676FEA811BBA,2017-07-13T22:05:51-07:00
6083,ad237e58eec063d15c0856131131c4e2,Horrifying immersive well written storytelling,I love S&S and Dark Topic. West side Fairytai...,5,F19703FACB7CA26,2019-02-22T13:06:34-07:00
6114,ad237e58eec063d15c0856131131c4e2,Horrifying immersive well written storytelling,I love S&S and Dark Topic. West side Fairytai...,5,F19703FACB7CA26,2019-02-22T13:06:34-07:00
13835,e8fb77c98183e85b6bf75e85da02892d,"If you like Myths and Legends, look no further",Myths and Legends’ Jason Weiser brings his mod...,5,213BE0AEEC056FB,2018-09-09T21:53:40-07:00
...,...,...,...,...,...,...
1946650,ae97c70120af155f0813f5fba880e60a,"“Chyna,Chyna,Chyna..”",The Biden administration is bought and paid fo...,5,1079EE1C386A625,2022-08-07T16:11:20-07:00
1956830,a0dcc13fd50bb25c866a8b9a2eca0f50,Racist,Continues to mispronounce foreign golfer’s nam...,1,3A41905452F889F,2022-07-26T03:52:22-07:00
1964345,f2970432aad718a000ebac10e48ae6b0,Awe Inspiring!,Between this podcast and Spooked my mind and l...,5,CBB1165EE6429D9,2022-05-27T10:50:56-07:00
1970374,b41b7a2e79c1f747d21babca3de1c184,Awesome Podcast!,"New listener! Really enjoy the podcast, the Tu...",5,0837294EC18BD32,2022-07-12T14:49:18-07:00


In [28]:
#merging
# merged_df = df_reviews.merge(df_podcasts, on='podcast_id', how='left')
# merged_df[merged_df['podcast_id'].isna()]

x=pd.read_sql_query(
    ''' 
      SELECT DISTINCT r.podcast_id,
             p.title AS Title
      FROM reviews r 
      LEFT JOIN podcasts p
      ON p.podcast_id = r.podcast_iD
      WHERE p.podcast_id IS NULL
    ''', con
)
x.count()

podcast_id    1523
Title            0
dtype: int64

Findings:

It appears that there are 1523 podcasts that are not present in the "podcasts" table. This could be due to the fact that the table was not updated with the relevant information for these specific podcasts during the creation of the database. This can happen when there is a mismatch between the data being added to the table and the source of data.

**Runs basic checks**

In [29]:
df_runs.sample(5)

,run_at,max_rowid,reviews_added
13,2022-11-03 21:12:30,5427586,22495
12,2022-10-01 21:56:38,5364248,16649
6,2022-04-20 01:03:30,3855531,36589
5,2021-10-11 05:18:23,3428923,16845
1,2021-06-06 21:34:36,3300773,13139


In [30]:
df_runs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   run_at         15 non-null     object
 1   max_rowid      15 non-null     int64 
 2   reviews_added  15 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 488.0+ bytes


We can see that run_at is a object but it should be time series and other features were numeric

In [31]:
df_runs.reviews_added.sum()

2025333

The table shows a record of updates made to the "reviews_df" table. The "reviews_added" column represents the number of reviews that were added to the table during each update

## 3.Exploratory Data Analysis

**Analysis of the categories**

checking the distinct categories

In [32]:
import plotly.express as px
df = pd.read_sql('SELECT * FROM categories', con)
df['category_updated'] = df['category'].apply(lambda x: 'Arts' if 'art' in x.lower() else
                                                           'Business' if 'business' in x.lower() else
                                                           'Culture' if 'culture' in x.lower() else
                                                           'Religion' if x in ['buddhism', 'hinduism', 'islam', 'christianity', 'judaism', 'religion-spirituality'] else
                                                           'Society' if 'society' in x.lower() else
                                                           'Fiction' if 'fict' in x.lower() else
                                                           'Education' if 'edu' in x.lower() else
                                                           'Health' if 'health' in x.lower() else
                                                           'Tv' if 'tv' in x.lower() else
                                                           'Comedy' if 'comedy' in x.lower() else
                                                           'Kids' if 'kids' in x.lower() else
                                                           'Leisure' if 'leisure' in x.lower() else
                                                           'News' if 'news' in x.lower() else
                                                           'Music' if 'music' in x.lower() else
                                                           'Science' if 'science' in x.lower() else
                                                           'Sports' if 'sport' in x.lower() else
                                                           x.capitalize() if x.isalpha() else x)

result = df.groupby(['category_updated']).agg({'category_updated': 'count'})
result.rename(columns={'category_updated': 'POPULARITY'}, inplace=True)
result = result.reset_index()
result = result.sort_values(by='POPULARITY', ascending=False)
result = result.reset_index(drop=True)

fig = px.bar(result, x='category_updated', y='POPULARITY')
fig.show()




In [37]:
fig.show(render='chrome')
fig.write_html("plot.html")


Findings:

The most common category of the podcast is **'culture'** and the less common category is **'government'** 

We can now check the average ratings of podcasts in each category.


In [ ]:
query = """
        select c.category_updated, round(avg(rating),2) ratings, count(*) num_reviews
        from reviews 
        join (select *,
              (case
                  when category like "%art%" then 'Arts'
                  when category like "%business%" then 'Business'
                  when category like "%culture%" then 'Culture'
                  when category = "buddhism" then 'Religion'
                  when category = "hinduism" then 'Religion'
                  when category = "islam" then 'Religion'
                  when category = "christianity" then 'Religion'
                  when category = "judaism" then 'Religion'
                  when category = "religion-spirituality" then 'Religion'
                  when category like "%society%" then "Society"
                  when category like "%Fict%" then "Fiction"
                  when category like "%Edu%" then "Education"
                  when category like "%Health%" then "Health"
                  when category like "%Tv%" then "Tv"
                  when category like "%Comedy%" then "Comedy"
                  when category like "%Kids%" then "Kids"
                  when category like "%Leisure%" then "Leisure"
                  when category like "%News%" then "News"
                  when category like "%Music%" then "Music"
                  when category like "%Science%" then "Science"
                  when category like "%sport%" then "sports"
                  else upper(substr(category, 1, 1)) || substr(category, 2)
              end) as category_updated
            FROM categories) c
        using (podcast_id)
        group by category_updated
        order by rating desc
        """
avg_ratings = pd.read_sql(query, con)
avg_ratings = avg_ratings.sort_values(by='ratings', ascending=False)
avg_ratings= avg_ratings.reset_index(drop=True)
avg_ratings


In [ ]:
fig = px.bar(avg_ratings, x='category_updated', y='ratings', color='num_reviews',log_y=True,
             title='Average Ratings by Category',
             labels={'category_updated': 'Category', 'average_rating': 'Average Rating', 'num_reviews': 'Number of Reviews'})
fig.show()


Findings:

We can see that podcasts of categories Spirituality and Businesss are rated the highest, while podcasts about News and Crime are rated the lowest.

Now we can see most popular podcasts

In [ ]:
query = """
        select c.category_updated,  count(*) as count
        from podcasts
        join (select *,
              (case
                  when category like "%art%" then 'Arts'
                  when category like "%business%" then 'Business'
                  when category like "%culture%" then 'Culture'
                  when category = "buddhism" then 'Religion'
                  when category = "hinduism" then 'Religion'
                  when category = "islam" then 'Religion'
                  when category = "christianity" then 'Religion'
                  when category = "judaism" then 'Religion'
                  when category = "religion-spirituality" then 'Religion'
                  when category like "%society%" then "Society"
                  when category like "%Fict%" then "Fiction"
                  when category like "%Edu%" then "Education"
                  when category like "%Health%" then "Health"
                  when category like "%Tv%" then "Tv"
                  when category like "%Comedy%" then "Comedy"
                  when category like "%Kids%" then "Kids"
                  when category like "%Leisure%" then "Leisure"
                  when category like "%News%" then "News"
                  when category like "%Music%" then "Music"
                  when category like "%Science%" then "Science"
                  when category like "%sport%" then "sports"
                  else upper(substr(category, 1, 1)) || substr(category, 2)
              end) as category_updated
            FROM categories) c
        using (podcast_id)
        group by 1
        order by 2 desc
        """
avg_ratings = pd.read_sql(query, con)

avg_ratings

In [ ]:
fig = px.bar(avg_ratings, x='category_updated', y='count',title='Popular Podcasts',)
fig.show()

Findings:

The most popular categories of the podcasts are Culture and Business

**Analysis of the Podcasts**

We can see now most populat tiltle with number of reviews and ratings

In [ ]:
query = """select p.title, count(*) num_reviews, avg(rating) average_rating
          from podcasts p
          join reviews r
          using (podcast_id)
          group by 1
          order by 2 desc
          limit 10
        """
most_reviews = pd.read_sql(query, con)
most_reviews

In [ ]:

fig = px.bar(most_reviews, x='title', y='num_reviews', color='average_rating',log_y=True,
             title='Podcasts with the Most Reviews',
             labels={'title': 'Podcast Title', 'num_reviews': 'Number of Reviews',
                     'average_rating': 'Average Rating'})
fig.show()


Findings:

It seems that "My Favourite Murder with Karen Kilgariff and Crime Junkie" is the most reviewd podcsat having more than twice reviews compared to other podcasts. 

We can also see the podcasts with highest number of rating with less counts 

In [ ]:
query = """select p.title, round(avg(rating), 2) rating, count(*) num_reviews
          from reviews r
          join podcasts p
          using (podcast_id)
          group by 1
          having num_reviews > 1000
          order by 2 desc
          limit 10
        """
top_podcasts= pd.read_sql(query, con)
top_podcasts

In [ ]:
fig = px.bar(top_podcasts.sort_values(by='rating', ascending=False),
             x='title', y='rating', color='num_reviews',
             log_y=True,
             title='Top Podcasts by Average Rating and Number of Reviews',
             labels={'title': 'Podcast Title', 'num_reviews': 'Number of Reviews',
                     'rating': 'Average Rating'})
fig.show()

"Shawn Ryan Show" and 'Sassy Lashes' seems to be the best rated podcast 

In [ ]:
best_podcasts=pd.read_sql_query('''
        SELECT p.title AS Title, AVG(r.rating) AS Average_rating, COUNT(r.rating) AS Number_of_votes
        FROM podcasts p
        JOIN reviews r
        ON p.podcast_id = r.podcast_id
        GROUP BY p.podcast_id
        ORDER BY 2 DESC
 



              ''', con)
best_podcasts.head(1)
best_podcasts.tail(1)

Now try to find out best and worst podcasts

In [ ]:
best_podcasts=pd.read_sql_query('''
        SELECT p.title AS Title, AVG(r.rating) AS Average_rating, COUNT(r.rating) AS Number_of_votes
        FROM podcasts p
        JOIN reviews r
        ON p.podcast_id = r.podcast_id
        GROUP BY p.podcast_id
        HAVING COUNT(r.rating) >= 500
        ORDER BY 2 DESC
        LIMIT 10



              ''', con)
best_podcasts

Now checking for the worst podcasts

In [ ]:
worst_podcasts=pd.read_sql_query('''
                  SELECT p.title AS Title, AVG(r.rating) AS Average_rating, COUNT(r.rating) AS Number_of_votes
                  FROM podcasts p
                  JOIN reviews r
                  ON p.podcast_id = r.podcast_id
                  GROUP BY p.podcast_id
                  HAVING COUNT(r.rating) >= 500
                  ORDER BY AVG(r.rating) ASC
                  LIMIT 10

                 ''', con)
worst_podcasts


In [ ]:
fig = px.bar(best_podcasts, x='Title', y='Average_rating', color='Average_rating', log_y=True,title='Best Podcasts')
fig2 = px.bar(worst_podcasts, x='Title', y='Average_rating', color='Average_rating', log_y=True, title='Worst Podcasts')

fig.show()
fig2.show()

Findings:


To have a good understanding of what podcast are mostly liked/disliked, I had to not only look at the average score of the reviews but also at the total number of voters. 

**Analysis of the Reviews**

Finding out about any reviewers voted more than 100 times?


In [ ]:
pd.read_sql_query( '''
       
                SELECT DISTINCT author_id As Author_ID,
                      COUNT(author_id) AS Number_of_reviews
                FROM reviews
                GROUP BY 1
                HAVING COUNT(author_id) >= 100
                ORDER BY 2 DESC
                ''', con)

Findings:

There are 51 bvery active reviewers on iTunes platform.

I would like to find out if specific author comented on a specific podcast more than once.

In [ ]:
pd.read_sql_query(
        '''
SELECT author_id, COUNT(author_id) AS Number_of_reviews, p.title AS Title, ROUND(AVG(rating), 2) AS Average_rating
FROM reviews r
JOIN podcasts p 
ON r.podcast_id = p.podcast_id
JOIN (SELECT podcast_id, GROUP_CONCAT(category, ', ') AS All_categories
      FROM categories
      GROUP BY podcast_id) g ON p.podcast_id = g.podcast_id
GROUP BY author_id
HAVING COUNT(author_id) >= 100
ORDER BY 2 DESC
 ''', con)


The authors may have left multiple reviews for various episodes of a podcast. However, the COUNT() function used in the query aggregates the number of reviews per author over the entire podcast title, not individual episodes. This is a limitation of the data set and the query

Reviews on specific time

In [ ]:
df_reviews.head(2)

In [ ]:
df=pd.read_sql_query(
        '''
SELECT 
  (CASE 
    WHEN CAST(SUBSTR(created_at, 12, 3) AS NUMERIC) BETWEEN 0 AND 6 THEN 'NIGHT'
    WHEN CAST(SUBSTR(created_at, 12, 3) AS NUMERIC) BETWEEN 6 AND 8 THEN 'EARLY MORNING'
    WHEN CAST(SUBSTR(created_at, 12, 3) AS NUMERIC) BETWEEN 8 AND 12 THEN 'MORNING'
    WHEN CAST(SUBSTR(created_at, 12, 3) AS NUMERIC) BETWEEN 12 AND 18 THEN 'AFTERNOON'
    WHEN CAST(SUBSTR(created_at, 12, 3) AS NUMERIC) BETWEEN 18 AND 20 THEN 'EARLY EVENING'
    ELSE 'EVENING'
  END) AS Part_of_the_day,
  COUNT(rating) AS Number_of_reviews,
  AVG(rating) AS Average_score
FROM reviews
GROUP BY Part_of_the_day
ORDER BY 2 DESC
 ''', con)
df

In [ ]:
fig = px.bar(df, x='Part_of_the_day', y='Number_of_reviews', color='Average_score',
             color_continuous_scale='Viridis')
fig.show()

Findings:

We can see here the reviewers are mostly active in afternoons to give the ratings

# **4.Inferential Statastics**

**4.1 CI for podcasts analysis based on business-careers**




In [ ]:
business_careers = pd.read_sql_query(
    '''
    WITH business_careers AS (
      SELECT podcast_id,
             category
      FROM categories
      WHERE category = 'business-careers'
    )
    SELECT DISTINCT r.content AS review,
           b.podcast_id AS ID,
           r.rating AS rating
    FROM business_careers b
    INNER JOIN reviews r
    ON r.podcast_id = b.podcast_id
    ''', con)
business_careers


In [ ]:
business_careers.describe()['rating']

In [ ]:
import scipy.stats as stats

mean = business_careers['rating'].mean()
stddev = business_careers['rating'].std()
n = len(business_careers)
t_critical = stats.t.ppf(0.975, n - 1) # two-tailed t-distribution
margin_of_error = t_critical * stddev / (n ** 0.5)

confidence_interval = (mean - margin_of_error, mean + margin_of_error)

confidence_interval

"The 95% confidence interval for the mean rating of podcasts in the 'business-careers' category is (4.8862, 4.8958). This means that we are 95% confident that the true mean rating for podcasts in this category falls between 4.8862 and 4.8958. The average rating for the podcasts in this category is 4.8867, which falls within the confidence interval."

**4.2 Retreving art related podcats with different groups**

In [ ]:
df_comb = pd.read_sql_query( '''
                            SELECT DISTINCT r.content AS review,
                            c.podcast_id AS ID,
                            c.category,
                            r.rating AS rating
                            FROM categories c
                            INNER JOIN reviews r
                            ON r.podcast_id = c.podcast_id
                            WHERE c.category IN ('arts-performing-arts', 'arts-visual-arts')
                                ''', con
                            )  
df_comb


In [ ]:
df_comb.groupby('category').describe()

For 'arts-performing-arts':

Mean = 4.705583

Standard deviation = 0.908338

For 'arts-visual-arts':

Mean = 4.822701

Standard deviation = 0.697645

 The formula for the t-interval with n-1 degrees of freedom, where n is the sample size, is:

 **mean ± t-critical * (standard deviation / √n)**

 Where t-critical is the critical value from the t-distribution table for the desired confidence level and degrees of freedom.



In [ ]:
import numpy as np
from scipy import stats

grouped = df_comb.groupby('category')
mean_performing_arts = grouped.get_group('arts-performing-arts').rating.mean()
mean_visual_arts = grouped.get_group('arts-visual-arts').rating.mean()

standard_deviation_performing_arts = grouped.get_group('arts-performing-arts').rating.std()
standard_deviation_visual_arts = grouped.get_group('arts-visual-arts').rating.std()

n_performing_arts = grouped.get_group('arts-performing-arts').rating.count()
n_visual_arts = grouped.get_group('arts-visual-arts').rating.count()

t_critical = stats.t.ppf(0.975, n_performing_arts-1)
confidence_interval_performing_arts = (mean_performing_arts - t_critical * (standard_deviation_performing_arts / (n_performing_arts**0.5)), 
                                        mean_performing_arts + t_critical * (standard_deviation_performing_arts / (n_performing_arts**0.5)))

t_critical = stats.t.ppf(0.975, n_visual_arts-1)
confidence_interval_visual_arts = (mean_visual_arts - t_critical * (standard_deviation_visual_arts / (n_visual_arts**0.5)), 
                                    mean_visual_arts + t_critical * (standard_deviation_visual_arts / (n_visual_arts**0.5)))

print(confidence_interval_performing_arts)
print(confidence_interval_visual_arts)


Findings:

Based on the confidence intervals for both arts-performing-arts and arts-visual-arts, we can say that with a 95% level of confidence, the true mean rating for arts-performing-arts lies between 4.6953 and 4.7159. And the true mean rating for arts-visual-arts lies between 4.8120 and 4.8334. These intervals provide a range of values that we expect the true mean to fall into.

It appears that the mean ratings for both arts categories are generally high, with a mean rating of 4.7056 for arts-performing-arts and 4.8227 for arts-visual-arts. These results suggest that listeners generally have a positive perception of podcasts in these categories.


**4.3 Hypothesis testing for business-careers podcasts**.




In [ ]:
business_careers.describe()['rating']

Based on the statistics provided, it looks like the mean rating is 4.89, with a standard deviation of 0.56. Most of the ratings seem to be 5, with a minimum of 1 and a maximum of 5.

To perform a hypothesis test, we need to specify the null and alternative hypotheses and choose a test statistic and significance level. Some possible hypothesis tests could be:

Test the mean rating against a specified value:

H0: The mean rating is equal to a specified value (e.g. 4.5)

Ha: The mean rating is not equal to the specified value

Test the proportion of 5-star ratings against a specified value:

H0: The proportion of 5-star ratings is equal to a specified value (e.g. 0.5)

Ha: The proportion of 5-star ratings is not equal to the specified value

Once the hypotheses have been defined, a test statistic (e.g. z-score or p-value) can be calculated and compared against a significance level (e.g. alpha = 0.05) to make a conclusion about the hypotheses.

In [ ]:
# Load the data into a numpy array
data = [4.890994, 0.562712, 53263, 5.000000]
mean, std, n, specified_value = data


z = (mean - specified_value) / (std / np.sqrt(n))

# Calculate the p-value
p_value = stats.norm.sf(np.abs(z)) * 2

# Compare the p-value to the significance level
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")


Findings:

Based on the results of the hypothesis test, we reject the null hypothesis that the mean rating is equal to the specified value (e.g. 4.5). This means that there is sufficient evidence to suggest that the mean rating is significantly different from the specified value.

In other words, the results of the test indicate that the mean rating for the business careers is not equal to the specified value, and therefore we can conclude that the business careers have a different mean rating. This conclusion is made based on a significance level of alpha = 0.05, meaning that there is a less than 5% chance of obtaining this result if the null hypothesis were true.

**4.4 Testing if difference in population mean is significant**

In [ ]:
df_comb.groupby('category').describe()

A hypothesis test is being conducted to determine if there is a significant difference between the average score of podcasts in the category "arts-performing-arts" and "arts-visual-arts". The null hypothesis is that there is no difference between the two means, while the alternative hypothesis is that there is a significant difference. The significance level is set at 5%.

Null: There is no difference between the average score of arts-performing-arts and arts-visuals-arts podcast.

Alternative: There is significant difference in mean average score.

Significance level - 5%.

In [ ]:
significance_level=f'(μ1 - μ2) = {round((4.70 - 4.83), 2)}'
f'significance_level is {significance_level}'



In [ ]:
import scipy.stats as sm

# Define the data for each category
arts_performing = df_comb.loc[df_comb.category == 'arts-performing-arts', "rating"]
arts_visual = df_comb.loc[df_comb.category == 'arts-visual-arts', "rating"]

# Perform the t-test
result = sm.stats.ttest_ind(arts_performing, arts_visual, equal_var=False)
print(result)

# Check the p-value
if result[1] < 0.05:
    print("There is significant difference in mean average score.")
else:
    print("There is no significant difference in mean average score.")



Findings:


The results of the t-test indicate that the p-value (5.39e-54) is less than the significance level (0.05), suggesting strong evidence against the null hypothesis. This means that there is a significant difference between the average scores of podcasts in the two categories.

In summary, based on the hypothesis test and its results, we can conclude that there is a significant difference in the mean average score between podcasts in the "arts-performing-arts" and "arts-visual-arts" categories.

# **5.Conclusion**

I used pandas and SQL to analyze iTunes podcasts and found that there are 20 categories available. 

Business-culture is the most popular category. 

I also found the most highly rated and least highly rated podcasts, as well as the most active reviewers and their number of reviews.

Through statistical analysis, I concluded that to reach a wide audience as a future podcaster, it would be best to focus on less serious topics and have a large number of episodes available.